# Notebook 03: Clean and Combine the Corpora

**Project: Data Triage of Transcribed Nixon Tapes** <br>
*Michelle Ballard and April Crompton* <br>
Loyola University Maryland Data Science Project 

## Import statements

In [1]:
import pickle # for serialization
import pprint
import re # for regex parsing
import sys

import pandas as pd
import openpyxl # to export files to Microsoft Excel
import io # to read Microsoft Excel files
from fuzzywuzzy import fuzz, process # to clean names

In [2]:
# unpickle preserved dataframes
pkl_file = open('cleaned_Watergate.pkl', 'rb')

df_all_wgtrial = pickle.load(pkl_file)
print("\n~~~df_all_wgtrial~~~\n")
print(df_all_wgtrial.info())

pkl_file2 = open('cleaned_WSPF.pkl', 'rb')

df_all_wspf = pickle.load(pkl_file2)
print("\n~~~df_all_wspf~~~\n")
print(df_all_wspf.info())


~~~df_all_wgtrial~~~

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, connally_exhibit_1 to exhibit_10
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   exhibit_number        31 non-null     object         
 1   conversation_number   31 non-null     object         
 2   cassette_number       31 non-null     object         
 3   abstract              31 non-null     object         
 4   location              31 non-null     object         
 5   participants          31 non-null     object         
 6   minutes               31 non-null     float64        
 7   txtdate               31 non-null     object         
 8   date                  31 non-null     datetime64[ns] 
 9   start_dtime           31 non-null     datetime64[ns] 
 10  end_dtime             31 non-null     datetime64[ns] 
 11  time_diff             31 non-null     timedelta64[ns]
 12  full_extracted_text   3

## Combine the Watergate Trial and Watergate Special Prosecution Force DataFrames

### Align the files

In [3]:
# create a key to identify WG Trial vs WSPF
df_all_wgtrial.insert(0, 'corpus', "WG_Trial")
df_all_wspf.insert(0, 'corpus', "WSPF")

# ensure the files align
# eliminate the exhibit_number from WG Trial (not needed)
df_all_wgtrial.drop(columns=['exhibit_number'], axis=1, inplace=True)
#        add blank abstract column to WSPF because these transcripts don't have an abstract
df_all_wspf['abstract'] = None

# concatenate the files
df_all_combined = pd.concat([df_all_wgtrial, df_all_wspf])

# eliminate the full_extracted_text column
df_all_combined.drop(columns=['full_extracted_text'], axis=1, inplace=True)

# reset the index, keep the current index as exhibit
df_all_combined.reset_index(inplace=True, drop=False)
df_all_combined.rename(columns={"index":"exhibit"}, inplace=True)

### Remove Duplicates

Found during iterative analysis, several exhibits cover the same conversations.  
These are removed in the dataframes below and the Watergate Trial Tapes version, which typically has less content, is deleted.  

WGT|WSPF
0,  33
1, 42 (42 has more text)
2, 43
3, 44 (44 has more text)
4, 46 (46 has slightly more text)
28, 47
5, 49 (49 has more text)
30, 60 (60 has more text)
6, 62
11, 68 (68 has more text)
12, 69 (69 has more text)
13, 72
15, 75 (75 has more text)
16, 76
17, 77 (77 has more text)
18, 78 (78 has more text)
20, 81 (81 has more text)
21, 82
22, 86 (86 has more text)
23, 87 (87 has more text)
24, 93 (93 has more text)
25, 101
26, 102
27, 106

In [4]:
df_all_combined.loc[[0,33,1,42, 2,43,3,44,4,46,28,47,5,49,30,60,6,62,11,68,12,69,13,72,
                     15,75,16,76,17,77,18,78,20,81,21,82,22,86,23,87,24,93,25,101,
                     26,102,27,106],['exhibit','start_dtime','end_dtime','abstract']]

,exhibit,start_dtime,end_dtime,abstract
0,connally_exhibit_1,1971-03-23 17:05:00,1971-03-23 17:38:00,A discussion of the Associated Milk Producers ...
33,472-021,1971-03-23 17:05:00,1971-03-23 17:38:00,None
1,exhibit_01,1972-06-23 10:04:00,1972-06-23 11:39:00,"""TheSmoking Gun"" conversation: Haldeman and Ni..."
42,741-002,1972-06-23 10:04:00,1972-06-23 11:39:00,None
2,exhibit_02,1972-06-23 13:04:00,1972-06-23 13:13:00,A discussion of how Hunt's involvement in the ...
43,741-010,1972-06-23 13:04:00,1972-06-23 13:13:00,None
3,exhibit_03,1972-06-23 14:20:00,1972-06-23 14:45:00,Haldeman discusses his meeting with Vernon Wal...
44,343-036,1972-06-23 14:20:00,1972-06-23 14:45:00,None
4,exhibit_04,1972-09-15 17:27:00,1972-09-15 18:17:00,A discussion of press treatment of the break-i...
46,779-002,1972-09-15 17:24:00,1972-09-15 18:17:00,None


### Add abstracts from duplicate WG Trial records to WSPF records
Where the Watergate Trial version is deleted, the abstract provided on the website is now added to the kept Watergate Special Prosecution Force record in order to salvage the available metadata.  
The transferred abstract is annotated to identify that it originated from the Watergate Trial version

In [5]:
duplicates = [0,33,1,42, 2,43,3,44,4,46,28,47,5,49,30,60,6,62,11,68,12,69,13,72,15,75,16,76,17,77,18,78,20,81,21,82,22,86,23,87,24,93,25,101,26,102,27,106]
deletes = duplicates[0::2] # evey even-indexed duplicate file will be deleted 
add_abs = duplicates[1::2] # every odd-indexed duplicate file will be kept 
print(deletes, "\n",add_abs) # review

[0, 1, 2, 3, 4, 28, 5, 30, 6, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27] 
 [33, 42, 43, 44, 46, 47, 49, 60, 62, 68, 69, 72, 75, 76, 77, 78, 81, 82, 86, 87, 93, 101, 102, 106]


In [6]:
# The abstract from the Watergate Trial version is transferred to the duplicate WSPF version
for i in range(len(deletes)):
    d = deletes[i]
    a = add_abs[i]
    abs = str("from WGTrialTapes "+df_all_combined['exhibit'][d]+" "+df_all_combined.abstract[d])
    df_all_combined.update(pd.DataFrame({'abstract':[abs]}, index=[a]))

# Review one of the updated files
display(df_all_combined.loc[106,:])

exhibit                                                   038-156_038-157
corpus                                                               WSPF
conversation_number                                               038-156
cassette_number                                                   [E-127]
abstract                from WGTrialTapes exhibit_33 Subjects covered ...
location                                            White House Telephone
participants            [President Nixon, H. R. ("Bob") Haldeman, Whit...
minutes                                                                 8
txtdate                                                    April 25, 1973
date                                                  1973-04-25 00:00:00
start_dtime                                           1973-04-25 19:46:00
end_dtime                                             1973-04-25 19:53:00
time_diff                                                 0 days 00:07:00
dropped                 [Unintelligibl

### Delete duplicate WG Trial Tapes records

In [7]:
# identify exhibit indexes to delete from df_speakers
exhibits_todelete = df_all_combined.loc[deletes,'exhibit'].values
exhibits_todelete

array(['connally_exhibit_1', 'exhibit_01', 'exhibit_02', 'exhibit_03',
       'exhibit_04', 'exhibit_35', 'exhibit_05', 'exhibit_10',
       'exhibit_11', 'exhibit_16', 'exhibit_17', 'exhibit_18',
       'exhibit_20', 'exhibit_21', 'exhibit_22', 'exhibit_23',
       'exhibit_25', 'exhibit_26', 'exhibit_27', 'exhibit_28',
       'exhibit_29', 'exhibit_31', 'exhibit_32', 'exhibit_33'],
      dtype=object)

In [8]:
# filter out WGTrial duplicate records
df_all_combined = df_all_combined[~df_all_combined['exhibit'].isin(exhibits_todelete)]

# Reset the indexes without adding a new column for index
df_all_combined.reset_index(inplace=True, drop=True)

# Review
df_all_combined.tail()

,exhibit,corpus,conversation_number,cassette_number,abstract,location,participants,minutes,txtdate,date,start_dtime,end_dtime,time_diff,dropped,speech_header,speech_flat_norepeat,speech_final
83,905-008,WSPF,905-008,"[E-128, E-129]",None,White House Oval Office,"[President Nixon, H. R. (""Bob"") Haldeman]",89,"April 26, 1973",1973-04-26,1973-04-26 08:55:00,1973-04-26 10:24:00,0 days 01:29:00,"[Unintelligible, unintelligible, unintelligibl...",TRANSCRIPT OF A MEETING BETWEEN THE PRESIDENT ...,PRESIDENT: Hi.\nHALDEMAN: Hi.\nHALDEMAN: Just ...,Hi. Hi. Just to show you somebody still has...
84,431-009,WSPF,431-009,"[E-130, E-131, E-135, E-136]",None,Old Executive Office Building,"[President Nixon, H. R. (""Bob"") Haldeman, Joh...",309,"April 26, 1973",1973-04-26,1973-04-26 15:59:00,1973-04-26 21:03:00,0 days 05:04:00,"[20 second pause, Unintelligible, unintelligib...",TRANSCRIPT OF A RECORDING OF A MEETING AMONG T...,"PRESIDENT: Bob, how’re ya doin'?\nHALDEMAN: Ok...","Bob, how’re ya doin'? Okay. Had a long day..."
85,442-001-069,WSPF,442-001,"[E-137, E-138, E-142, E-143]",None,Old Executive Office Building,"[President Nixon, John W. Dean, III, Stephen B...",281,"June 4, 1973",1973-06-04,1973-06-04 00:00:00,1973-06-04 00:00:00,0 days 00:00:00,"[Noise, 33, 684, Noise, Pause. Bull enters, u...",*** DRAFT *** TRANSCRIPT PREPARED BY THE IMP...,"PRESIDENT: Bull, please. Steve. Hello. Than...","Bull, please. Steve. Hello. Thank you. I..."
86,039-080_039-081,WSPF,039-080,[E-144],None,White House Telephone,"[President Nixon, H. R. (""Bob"") Haldeman, Whit...",15,"June 4, 1973",1973-06-04,1973-06-04 22:05:00,1973-06-04 22:20:00,0 days 00:15:00,"[Unintelligible, CONTINUED, unintelligible, un...",TRANSCRIPT OF A RECORDING OF A TELEPHONE CONVE...,"OPERATOR: Yes, please.\nPRESIDENT: Mr. Haldema...","Yes, please. Mr. Haldeman, please. Thank yo..."
87,039-083,WSPF,039-083,[E-145],None,White House Telephone,"[President Nixon, H. R. (""Bob"") Haldeman]",1,"June 4, 1973",1973-06-04,1973-06-04 22:21:00,1973-06-04 22:22:00,0 days 00:01:00,[],TRANSCRIPT OF A TELEPHONE CONVERSATION BETWEEN...,PRESIDENT: Bob?\nOPERATOR: Just a moment. Her...,Bob? Just a moment. Here you are. Sorry to...


## Add a column of expanded contractions

In [9]:
CONTRACTION_MAP = {
"&": "and",
"#": "number",
"\.\.+": " ",
"--+": " ",
"\*+": " ",
r"(?<=[A-Za-z])-(?=[A-Za-z])": "_",
"they ain't": "they are not",
"ain't": "is not",
r"o'clock": "oclock",
r"can't": "can not",
r"(?<=\s)'cause": "because",
r"(?<=[A-Za-z]in)'": "g",
r"(?<=[A-Za-z])n't": " not",
r"(?<=[A-Za-z])'re": " are",
r"(?<=[A-Za-z])'s": " is",
r"(?<=[A-Za-z])'d": " would",
r"(?<=[A-Za-z])'ll": " will",
r"(?<=[A-Za-z])'t": " not",
r"(?<=[A-Za-z])'ve": " have",
r"(?<=[A-Za-z])'m": " am"
}


def expanded(rs, cdict=CONTRACTION_MAP):
    """This function expands contractions within text"""
    for key, value in cdict.items():
        rs = re.sub(key, value, rs)
    return rs

In [10]:
# Create expanded contractions column
df_all_combined['speech_final_expanded'] = \
df_all_combined.speech_final.copy().apply(lambda row: expanded(row))

In [11]:
# review the updated dataframe
print(df_all_combined.info())
display(df_all_combined.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   exhibit                88 non-null     object         
 1   corpus                 88 non-null     object         
 2   conversation_number    88 non-null     object         
 3   cassette_number        88 non-null     object         
 4   abstract               31 non-null     object         
 5   location               88 non-null     object         
 6   participants           88 non-null     object         
 7   minutes                88 non-null     object         
 8   txtdate                88 non-null     object         
 9   date                   88 non-null     datetime64[ns] 
 10  start_dtime            88 non-null     datetime64[ns] 
 11  end_dtime              88 non-null     datetime64[ns] 
 12  time_diff              88 non-null     timedelta64[n

,exhibit,corpus,conversation_number,cassette_number,abstract,location,participants,minutes,txtdate,date,start_dtime,end_dtime,time_diff,dropped,speech_header,speech_flat_norepeat,speech_final,speech_final_expanded
0,exhibit_12,WG_Trial,886-8,8 E - 7 Segment 1 9 E - 8 Segment 1 10...,"The ""Cancer on the Presidency"" conversation: D...",White House Oval Office,"[President Nixon, H.R.Haldeman, John W. Dean III]",83.0,"Wednesday, March 21, 1973",1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00,"[Sighs, unintelligible, Background noises, Uni...",TRANSCRIPT OF A RECORDING OF A MEETING AMONG T...,"PRESIDENT: John, sit down, sit down.\nDEAN: Go...","John, sit down, sit down. Good morning. Wel...","John, sit down, sit down. Good morning. Wel..."
1,exhibit_13,WG_Trial,421-18,E - 10 Segment 1,Dean advises the President about Hunt's involv...,Old Executive Office Building Office,"[President Nixon, H.R.Haldeman, John W. Dean I...",36.0,"Wednesday, March 21, 1973",1973-03-21,1973-03-21 17:20:00,1973-03-21 18:01:00,0 days 00:41:00,"[Noise, Unintelligible, unintelligible overlai...",TRANSCRIPT OF A RECORDING OF A MEETING AMONG T...,"PRESIDENT: (Unintelligible)\nEHRLICHMAN: Well,...","Well, you go round and round and you come u...","Well, you go round and round and you come u..."
2,exhibit_14,WG_Trial,37-204,E - 11 Segment 1,Topics discussed: Colson's insistence that Wat...,White House Telephone,"[President Nixon, Charles W. Colson]",5.0,"Wednesday, March 21, 1973",1973-03-21,1973-03-21 19:53:00,1973-03-21 20:24:00,0 days 00:31:00,"[unintelligible, clears throat, clears throat,...",TRANSCRIPT OF A TELEPHONE CONVERSATION BETWEEN...,"COLSON: Well, he said, anyway. But you know, h...","Well, he said, anyway. But you know, he, that...","Well, he said, anyway. But you know, he, that..."
3,exhibit_15,WG_Trial,422-20,E - 11 Segment 2,Subjects covered include: the possible use of ...,Old Executive Office Building Office,"[President Nixon, H.R.Haldeman]",31.0,"Thursday, March 22, 1973",1973-03-22,1973-03-22 09:11:00,1973-03-22 10:35:00,0 days 01:24:00,"[unintelligible, unintelligible, Unintelligibl...",TRANSCRIPT OF A RECORDING OF A MEETING BETWEEN...,"HALDEMAN: Well, the theory, the theory of the ...","Well, the theory, the theory of the Ervin ana...","Well, the theory, the theory of the Ervin ana..."
4,exhibit_19,WG_Trial,896-4,E - 17 Segment 1,This conversation recounts Ehrlichman's meetin...,White House Oval Office,"[President Nixon, H.R.Haldeman, John D. Ehrlic...",39.0,"Saturday, April 14, 1973",1973-04-14,1973-04-14 14:24:00,1973-04-14 15:55:00,0 days 01:31:00,"[Unintelligible, Unintelligible, Unintelligibl...",TRANSCRIPT OF A RECORDING OF A MEETING AMONG T...,PRESIDENT: All finished?\nEHRLICHMAN: Yes sir....,All finished? Yes sir. All finished. He is...,All finished? Yes sir. All finished. He is...


## Create the Speaker DataFrame

### Set up the Speaker Dataframe

In [12]:
# speaker text function to put speaker and text into a dataframe

def speaker_text(exhibit_speech):
    """this function identifies speakers by the capital-letters-colon pattern,
    uses the start and end position of a speaker name to segment the document by
    speaker and what was said in chronological order"""
    # find start/end of speaker
    speaker_pattern1 = r'[A-Z]+\b:' 
    speaker_pattern2 = r'[A-Z][A-Za-z0-9\']+\b:'#@Misha: updated to account for camelcase
    p = re.compile(speaker_pattern1)
    speakers = p.findall(exhibit_speech)
    if len(speakers)<=3:                #@Misha: added the if-statement to account for camelcase
        p = re.compile(speaker_pattern2)
        speakers = p.findall(exhibit_speech)
    iterator = p.finditer(exhibit_speech)
    spans = []
    for match in iterator:
        spans.append(match.span())
    # capture speaker and speech until next speaker in a DataFrame    
    df_tmp = pd.DataFrame(columns=["exhibit","speaker","para"])
    for i in range(len(spans)):
        speaker_start = spans[i][0]
        text_start = spans[i][1]
        text_end = (spans[i+1][0]) if (i+1) < len(spans) else -1
        speaker_tmp = exhibit_speech[speaker_start:text_start]
        speaker_tmp = re.sub('[^A-Za-z0-9]+', '', speaker_tmp)
        text_tmp = exhibit_speech[text_start:text_end]
        a = pd.DataFrame([[speaker_tmp, text_tmp]], columns=["speaker","para"])
        df_tmp = pd.concat([df_tmp,a], ignore_index=True)

    return df_tmp

In [13]:
%%time

# Loop through all exhibits, creating an indexed dataframe of speakers per exhibit
df_speakers = pd.DataFrame(columns=["exhibit","speaker","para"])

spkr_text = df_all_combined[['exhibit','speech_flat_norepeat']].copy()
for i in range(len(spkr_text)):
    if isinstance(spkr_text.speech_flat_norepeat[i], (list, tuple)):
        st = ' '.join(spkr_text.speech_flat_norepeat[i]) # Allows the loop to run on strings or lists by flattening the list one level
    else: st = spkr_text.speech_flat_norepeat[i]
    df_temp = speaker_text(st)
    df_temp["exhibit"] = spkr_text.exhibit[i]
    df_speakers = pd.concat([df_speakers, df_temp], ignore_index=True)

# Review
df_speakers

CPU times: user 35.2 s, sys: 142 ms, total: 35.3 s
Wall time: 35.3 s


,exhibit,speaker,para
0,exhibit_12,PRESIDENT,"John, sit down, sit down.\n"
1,exhibit_12,DEAN,Good morning.\n
2,exhibit_12,PRESIDENT,"Well, what is the Dean summary of the day\nab..."
3,exhibit_12,DEAN,John caught me on the way out and asked me\na...
4,exhibit_12,PRESIDENT,Full committee.\n
...,...,...,...
34842,039-083,HALDEMAN,...sounded pretty good when I wrote them out\...
34843,039-083,PRESIDENT,"Fine, well, we just, uh, if we could preempt\..."
34844,039-083,HALDEMAN,Yep.\n
34845,039-083,PRESIDENT,"Okay, boy.\n"


### Clean Speaker Names

In [14]:
# Review issues
unique_spk = df_speakers['speaker'].value_counts()
print(len(unique_spk))
display(unique_spk.sort_index())

# Ref: https://www.geeksforgeeks.org/how-to-do-fuzzy-matching-on-pandas-dataframe-column-using-python/

190


AFFELDT       1
ALAGIA       10
ALL           1
AN            3
BESSEN        1
           ... 
WHITAKER      1
WHITE         1
WOODS         1
ZIECLER       1
ZIEGLER     538
Name: speaker, Length: 190, dtype: int64

In [15]:
# Build a list of expected speakers

# Review known speakers
unique_participants = []
unique_participants.extend(p for part in df_all_combined['participants'] for p in part if len(p)>0) #flatten
unique_participants = set(unique_participants) #get unique values
print("Participants identified:",len(unique_participants))
unique_participants = list(unique_participants)
display(unique_participants)

Participants identified: 31


['Charles W. Colson',
 'Leaders of the Dairy Industry',
 'White House Operator',
 'President Nixon',
 'Alexander M. Haig, Jr.',
 'Richard G. Kleindienst',
 'L. Patrick Gray',
 'Stephen B. Bull',
 'George P. Shultz',
 'John W. Dean, III',
 'E. Howard Hunt',
 'John W. Dean III',
 'Clifford M. Hardin',
 'Herbert W. Kalmbach',
 'H.R.Haldeman',
 'Alexander P. Butterfield',
 'John W, Dean, III',
 'John D, Ehrlichman',
 'William P. Rogers',
 'John T. Whitaker',
 'Rose Mary Woods',
 'Henry E. Petersen',
 'H. R. ("Bob") Haldeman',
 ' John D. Ehrlichman',
 'Manolo Sanchez',
 'John D. Ehrlichman',
 'Ronald L. Ziegler',
 'Richard A. Moore',
 'John N. Mitchell',
 ' H. R. ("Bob") Haldeman',
 'John B. Connally']

In [16]:
# Retrieve last names from Participant metadata category in uppercase
sep = [".",",","^","-","_","&","#",":"]
suffixes = [" jr", " sr", " 2nd", " c3rd", " 3rd"," iv", " vi", " v", " iii", " ii"] # note, order is critical; iii before ii, vi before f
up = []
for u in unique_participants:
    u = u.lower()
    for s in sep:
        u = u.replace(s," ") # space for any separator
    for x in suffixes:
        u = u.replace(x,"") # eliminate name suffixes (note, space is needed)
    " ".join(u.split()) # eliminate duplicate spaces
    up.append(u)
up_unique = [u.split() for u in up]
up_unique = [u[-1].upper() for u in up_unique] # this is the last name

# Add any other names that appear more than 5 times in the recorded speakers
#    Note: Finds "ZIEGLER", UNIDENTIFIED" and "VOICES"
more_spk = unique_spk[unique_spk>5].index.tolist()
more_spk = [m.upper() for m in more_spk] # Ensure all names are uppercase

up_unique.extend(more_spk) # combine Speakers with Participants
up_unique = list(set(up_unique)) # unique values only
up_unique.sort()
print("Speaker comparison list:",len(up_unique))
up_unique

#   Suffixes, inherited ref: https://mediawiki.middlebury.edu/LIS/Name_Standards#Suffix

Speaker comparison list: 40


['ALAGIA',
 'BULL',
 'BUTTERFIELD',
 'CAMPBELL',
 'COLSON',
 'CONNALLY',
 'CONT',
 'DEAN',
 'EHRLICHHAN',
 'EHRLICHMAN',
 'GRAY',
 'GRIFFIN',
 'HAIG',
 'HALDEMAN',
 'HARDIN',
 'HUNT',
 'INDUSTRY',
 'KALMBACH',
 'KLEINDIENST',
 'MAN',
 'MITCHELL',
 'MOORE',
 'NELSON',
 'NIXON',
 'OPERATOR',
 'PARR',
 'PETERSEN',
 'PPESIDENT',
 'PRESIDENT',
 'ROGERS',
 'SANCHEZ',
 'SECRETARY',
 'SHULTZ',
 'TALKING',
 'UNIDENTIFIED',
 'VOICES',
 'WAITER',
 'WHITAKER',
 'WOODS',
 'ZIEGLER']

In [17]:
# Manage known exceptions
up_unique.remove("EHRLICHHAN")
up_unique.remove("PPESIDENT")
print("Speaker comparison list:",len(up_unique))
up_unique

Speaker comparison list: 38


['ALAGIA',
 'BULL',
 'BUTTERFIELD',
 'CAMPBELL',
 'COLSON',
 'CONNALLY',
 'CONT',
 'DEAN',
 'EHRLICHMAN',
 'GRAY',
 'GRIFFIN',
 'HAIG',
 'HALDEMAN',
 'HARDIN',
 'HUNT',
 'INDUSTRY',
 'KALMBACH',
 'KLEINDIENST',
 'MAN',
 'MITCHELL',
 'MOORE',
 'NELSON',
 'NIXON',
 'OPERATOR',
 'PARR',
 'PETERSEN',
 'PRESIDENT',
 'ROGERS',
 'SANCHEZ',
 'SECRETARY',
 'SHULTZ',
 'TALKING',
 'UNIDENTIFIED',
 'VOICES',
 'WAITER',
 'WHITAKER',
 'WOODS',
 'ZIEGLER']

In [18]:
%%time

# Improve accuracy and consistency of speaker names:

# use Fuzzy Wuzzy to find a match for bad names
correct = []
similarity = []

for i in df_speakers['speaker']:
        ratio = process.extract(i, up_unique, limit=1)
        correct.append(ratio[0][0])
        similarity.append(ratio[0][1])

# Update with most similar option from cleaned list
df_speakers['speaker_clean'] = pd.Series(correct)
df_speakers['similarity'] = pd.Series(similarity)

# Make all PRESIDENT = NIXON for consistency
df_speakers['speaker_clean'].replace(
    to_replace="PRESIDENT", value="NIXON", inplace=True)

# Keep original data when similarity is <70%; indicate with similarity 0
df_updt = df_speakers[df_speakers['similarity']<70][['speaker','speaker_clean','similarity']]
display(df_updt)

df_updt['speaker_clean'].update(df_updt['speaker'].apply(lambda x: x.upper()))
df_updt['similarity'] = 0
display(df_updt)

df_speakers.update(df_updt)

display(df_speakers[df_speakers['similarity'] < 100][0:1000].sort_values(
    by='similarity', ascending=True))

#  Ref: https://medium.com/analytics-vidhya/matching-messy-pandas-columns-with-fuzzywuzzy-4adda6c7994f

,speaker,speaker_clean,similarity
355,IDEAL,DEAN,67
2743,HANSON,NELSON,67
2754,BESSEN,NIXON,60
2761,BUTTERBRODT,BUTTERFIELD,64
2770,AFFELDT,BUTTERFIELD,56
2772,BISHOP,NIXON,36
3055,GREGG,GRAY,44
3058,GREGG,GRAY,44
3060,GREGG,GRAY,44
3062,GREGG,GRAY,44


,speaker,speaker_clean,similarity
355,IDEAL,IDEAL,0
2743,HANSON,HANSON,0
2754,BESSEN,BESSEN,0
2761,BUTTERBRODT,BUTTERBRODT,0
2770,AFFELDT,AFFELDT,0
2772,BISHOP,BISHOP,0
3055,GREGG,GREGG,0
3058,GREGG,GREGG,0
3060,GREGG,GREGG,0
3062,GREGG,GREGG,0


,exhibit,speaker,para,speaker_clean,similarity
3481,472-021,RICE,"the rate,\n",RICE,0.0
2772,051-001,BISHOP,"John Bishop, Mr. President. Iowa.\n",BISHOP,0.0
3874,002-001_002-002,KLEUBDUEBST,(Laughs) Yeah. Yeah.\n,KLEUBDUEBST,0.0
2770,051-001,AFFELDT,Paul Affeldt [unintelligible]\n,AFFELDT,0.0
8643,862-004,Unknown,(Unintelligible)\n,UNKNOWN,0.0
...,...,...,...,...,...
14726,422-033,UNIIDENTIFIED,"No. They, uh --\n",UNIDENTIFIED,96.0
25067,429-003,UNINDENTIFIED,(Unintelligible)\n,UNIDENTIFIED,96.0
17061,428-019,UNIDENITIFIED,Right.\n,UNIDENTIFIED,96.0
13277,421-018,UNINDENTIFIED,That's a fact. We're above that.\n,UNIDENTIFIED,96.0


CPU times: user 36.2 s, sys: 62.5 ms, total: 36.2 s
Wall time: 36.3 s


### Groom text and identify non-speech

In [19]:
# functions for initial grooming

# prints all text contained in () for visual examination prior to removing
def examine_drops(text):
    regex = re.compile(".*?[\[\(](.*?)[\]\)]") # brackets are used instead of parentheses most of the time
    result = re.findall(regex, text)
    return(result)

# removes new line characters and all () including text
def transcript_initialgroom(text):
    #text = re.sub(r"\n", " ", text)
    text = re.sub(r"\s", " ", text)
    text = re.sub(r'\\\\',"",text)
    text = re.sub(r'\\',"",text)
    text = re.sub("[\[\(](.*?)[\]\)]", "", text) #updated with new drops pattern
    return(text)

In [20]:
%%time

# Drop text that is not useful dialogue, store dropped text
df_speakers['nonspeech'] = df_speakers['para'].apply(lambda row: examine_drops(row))

# Specify the interesting non-speech noises
df_speakers['nonspeech_clean'] = df_speakers['nonspeech'].apply(
    lambda x: list(filter(lambda a: 'unintelligible' not in a.lower(), x)))

# apply grooming to speech (removes new line characters and all () including text
df_speakers['speech_final'] = df_speakers['para'].apply(lambda row: transcript_initialgroom(row))

CPU times: user 1.43 s, sys: 12.6 ms, total: 1.45 s
Wall time: 1.45 s


In [21]:
# Create expanded contractions column
df_speakers['speech_final_expanded'] = \
df_speakers.speech_final.copy().apply(lambda row: expanded(row))

### Groom the dataframe

In [22]:
# Add useful metadata from the df_all file
df_speakers = pd.merge(df_speakers, df_all_combined.loc[:,["corpus","exhibit","location","date",
                                                           "start_dtime","end_dtime","time_diff"]],
                       how='left', left_on='exhibit', right_on='exhibit', sort=False)
df_speakers.head()

,exhibit,speaker,para,speaker_clean,similarity,nonspeech,nonspeech_clean,speech_final,speech_final_expanded,corpus,location,date,start_dtime,end_dtime,time_diff
0,exhibit_12,PRESIDENT,"John, sit down, sit down.\n",NIXON,100.0,[],[],"John, sit down, sit down.","John, sit down, sit down.",WG_Trial,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
1,exhibit_12,DEAN,Good morning.\n,DEAN,100.0,[],[],Good morning.,Good morning.,WG_Trial,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
2,exhibit_12,PRESIDENT,"Well, what is the Dean summary of the day\nab...",NIXON,100.0,[],[],"Well, what is the Dean summary of the day abo...","Well, what is the Dean summary of the day abo...",WG_Trial,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
3,exhibit_12,DEAN,John caught me on the way out and asked me\na...,DEAN,100.0,[],[],John caught me on the way out and asked me ab...,John caught me on the way out and asked me ab...,WG_Trial,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
4,exhibit_12,PRESIDENT,Full committee.\n,NIXON,100.0,[],[],Full committee.,Full committee.,WG_Trial,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00


In [23]:
# Select df_speakers columns
df_speakers = df_speakers[['corpus','exhibit','speaker_clean','nonspeech_clean',
                'speech_final','speech_final_expanded','location','date','start_dtime','end_dtime','time_diff']]
# Review outcome
display(df_speakers.info())
pd.options.display.max_rows = 1000
display(df_speakers[df_speakers['nonspeech_clean'].str.len() > 0][0:1000])
pd.reset_option("display.max_rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34847 entries, 0 to 34846
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   corpus                 34847 non-null  object         
 1   exhibit                34847 non-null  object         
 2   speaker_clean          34847 non-null  object         
 3   nonspeech_clean        34847 non-null  object         
 4   speech_final           34847 non-null  object         
 5   speech_final_expanded  34847 non-null  object         
 6   location               34847 non-null  object         
 7   date                   34847 non-null  datetime64[ns] 
 8   start_dtime            34847 non-null  datetime64[ns] 
 9   end_dtime              34847 non-null  datetime64[ns] 
 10  time_diff              34847 non-null  timedelta64[ns]
dtypes: datetime64[ns](3), object(7), timedelta64[ns](1)
memory usage: 3.2+ MB


None

,corpus,exhibit,speaker_clean,nonspeech_clean,speech_final,speech_final_expanded,location,date,start_dtime,end_dtime,time_diff
16,WG_Trial,exhibit_12,NIXON,[Sighs],I hope to God that we can get off though to...,I hope to God that we can get off though to...,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
24,WG_Trial,exhibit_12,NIXON,[Background noises],We are standing for the rights of innocent i...,We are standing for the rights of innocent i...,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
41,WG_Trial,exhibit_12,DEAN,"[1, 2]","I think, I think that, uh, there's no doubt a...","I think, I think that, uh, there is no doubt ...",White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
59,WG_Trial,exhibit_12,DEAN,[coughs],... and things like this. He'd worked with ...,and things like this. He would worked wit...,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
61,WG_Trial,exhibit_12,DEAN,"[Background noises, Clears throat, Clears throat]","January of '72. Like, ""You come over to Mit...","January of '72. Like, ""You come over to Mit...",White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
71,WG_Trial,exhibit_12,DEAN,[1],"Uh, Mitchell, Mitchell just virtually sat the...","Uh, Mitchell, Mitchell just virtually sat the...",White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
85,WG_Trial,exhibit_12,DEAN,[coughs],Now --,Now,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
87,WG_Trial,exhibit_12,DEAN,"[Clears Throat, 1]","Now. . So, Liddy went back after that and i...","Now. . So, Liddy went back after that and i...",White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
101,WG_Trial,exhibit_12,DEAN,[noise],"I'll go back over that, and tell you where I...","I will go back over that, and tell you where...",White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00
110,WG_Trial,exhibit_12,NIXON,[Sighs],yeah.,yeah.,White House Oval Office,1973-03-21,1973-03-21 10:12:00,1973-03-21 11:55:00,0 days 01:43:00


## Create a dataframe of ~30K character chunks per exhibit

In [24]:
%%time
# Create a dataframe for the chunks
df_chunks = pd.DataFrame(columns = ['corpus', 'orig_exhibit', 'exhibit', 'speech_final', 'abstract'])

# Build the chunks
for i,e in enumerate(df_all_combined.exhibit):
    # Publish it as-is if it is below 32K characters
    if len(df_all_combined.speech_final[df_all_combined.exhibit==e].values[0]) < 32000:
        d = pd.DataFrame.from_dict({'corpus' : [df_all_combined.corpus[i]], 'orig_exhibit' : [e], 
                                   'exhibit' : [e], 'speech_final' : [df_all_combined.speech_final[i]], 
                                   'abstract' : [df_all_combined.abstract[i]]})
        df_chunks = pd.concat([df_chunks,d], ignore_index=True)
    # Or chunk it into segments just over 30K
    else:
        exhibit_speech_chunk = str()
        exhibit_chunk = df_speakers[df_speakers.exhibit==e].index
        e_row_b = exhibit_chunk[0]
        e_row_e = exhibit_chunk[-1]+1 # add 1 so the range includes the last record
        e_num = 1
        while e_row_b < e_row_e:
            e_name = "Part_" + str(e_num) + "_" + e
            while len(exhibit_speech_chunk) < 30000:
                exhibit_speech_chunk = exhibit_speech_chunk +' '+ str(df_speakers.speech_final[e_row_b]).strip()
                e_row_b += 1
                if e_row_b == e_row_e: break
            d = pd.DataFrame.from_dict({'corpus' : [df_all_combined.corpus[i]], 'orig_exhibit' : [e], 
                                       'exhibit' : [e_name], 'speech_final' : [exhibit_speech_chunk], 
                                       'abstract' : [df_all_combined.abstract[i]]})
            df_chunks = pd.concat([df_chunks,d], ignore_index=True)
            exhibit_speech_chunk = str()
            e_num += 1
    
display(df_chunks) 

,corpus,orig_exhibit,exhibit,speech_final,abstract
0,WG_Trial,exhibit_12,Part_1_exhibit_12,"John, sit down, sit down. Good morning. Well,...","The ""Cancer on the Presidency"" conversation: D..."
1,WG_Trial,exhibit_12,Part_2_exhibit_12,"Sure. Uh, there's no doubt about that. Mrs. ...","The ""Cancer on the Presidency"" conversation: D..."
2,WG_Trial,exhibit_12,Part_3_exhibit_12,"Uh, he John doesn't go until Friday. Friday.....","The ""Cancer on the Presidency"" conversation: D..."
3,WG_Trial,exhibit_12,Part_4_exhibit_12,"But, based on what information it would? For ...","The ""Cancer on the Presidency"" conversation: D..."
4,WG_Trial,exhibit_13,exhibit_13,"Well, you go round and round and you come u...",Dean advises the President about Hunt's involv...
...,...,...,...,...,...
136,WSPF,442-001-069,Part_2_442-001-069,"No, I'm not going to bother with them. I ha...",None
137,WSPF,442-001-069,Part_3_442-001-069,"The, uh, FBI clips? I have that morning? They...",None
138,WSPF,442-001-069,Part_4_442-001-069,Good Which is very good. And then we take ou...,None
139,WSPF,039-080_039-081,039-080_039-081,"Yes, please. Mr. Haldeman, please. Thank yo...",None


CPU times: user 442 ms, sys: 7.97 ms, total: 450 ms
Wall time: 450 ms


In [25]:
pd.set_option('display.max_columns',800)
display(df_chunks.speech_final[df_chunks.orig_exhibit=="741-002"].iloc[0])

' I\'ve told that you\'ve got, they\'ve got magnificent photos No, they don\'t.  See, that was all hand held camera without lighting.  Lousy footage. It\'s good in content, it\'s terrible in film quality. Well, you\'ll have Rose there.    You know she deserves to be seen in here, she ought to be in here. Well, no problem.  Run her in if you want to sure That\'s right.  Got so God damned much God damned. I understand, I just thought  If I do, I just buzz. Yeah.  Ah Good, that\'s a very good paper that he prepared.  At least it volunteered one thing they haven\'t got in there is the thing we mentioned with regard to the Armed Services. I covered that with Ehrlichman who says that can be done and he\'s moving.  Not only Armed Services, but the whole government. GSA? All government? All government procurement, yeah.  And,, ah, I talked to John about that and he said that was a good idea.  So, ah, Henry gets back at 3:45. Yeah, I told, ah, Haig today I\'d see Rogers at 4:30. Oh.  good, O.K.

In [26]:
print(df_all_combined.speech_final[df_all_combined.exhibit=="741-002"].iloc[0])
pd.reset_option('display.max_columns')

  I've told that you've got, they've got magnificent photos  No, they don't.  See, that was all hand held camera without lighting.  Lousy footage. It's good in content, it's terrible in film quality.   Well, you'll have Rose there.    You know she deserves to be seen in here, she ought to be in here.  Well, no problem.  Run her in if you want to sure  That's right.  Got so God damned much   God damned.  I understand, I just thought  If I do, I just buzz.  Yeah.  Ah  Good, that's a very good paper that he prepared.  At least it volunteered one thing they haven't got in there is the thing we mentioned with regard to the Armed Services.  I covered that with Ehrlichman who says that can be done and he's moving.  Not only Armed Services, but the whole government.   GSA? All government?  All government procurement, yeah.  And,, ah, I talked to John about that and he said that was a good idea.  So, ah, Henry gets back at 3:45.  Yeah, I told, ah, Haig today I'd see Rogers at 4:30.  Oh.  good, 

In [27]:
# Create expanded contractions column
df_chunks['speech_final_expanded'] = \
df_chunks.speech_final.copy().apply(lambda row: expanded(row))

## Store Data
### Pickle the combined DataFrames

In [28]:
# pickle initial dataframe to avoid processing datagrooming
output = open('cleaned_combined.pkl', 'wb')
sys.setrecursionlimit(100000)

pickle.dump(df_all_combined, output)
pickle.dump(df_speakers, output)
pickle.dump(df_chunks, output)

output.close()

### Pickle import block:

In [29]:
import pprint, pickle
import pandas as pd

# unpickle preserved dataframes to continue EDA and Statistical Analysis
pkl_file = open('cleaned_combined.pkl', 'rb')

df_all_combined = pickle.load(pkl_file)
print("\n~~~df_all_combined~~~\n")
print(df_all_combined.info())

df_speakers_combined = pickle.load(pkl_file)
print("\n~~~df_speakers_combined~~~\n")
print(df_speakers_combined.info())

df_chunks_combined = pickle.load(pkl_file)
print("\n~~~df_chunks_combined~~~\n")
print(df_chunks_combined.info())

# Expected Results:

# ~~~df_all_combined~~~

# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 88 entries, 0 to 87
# Data columns (total 18 columns):
#  #   Column                 Non-Null Count  Dtype          
# ---  ------                 --------------  -----          
#  0   exhibit                88 non-null     object         
#  1   corpus                 88 non-null     object         
#  2   conversation_number    88 non-null     object         
#  3   cassette_number        88 non-null     object         
#  4   abstract               31 non-null     object         
#  5   location               88 non-null     object         
#  6   participants           88 non-null     object         
#  7   minutes                88 non-null     object         
#  8   txtdate                88 non-null     object         
#  9   date                   88 non-null     datetime64[ns] 
#  10  start_dtime            88 non-null     datetime64[ns] 
#  11  end_dtime              88 non-null     datetime64[ns] 
#  12  time_diff              88 non-null     timedelta64[ns]
#  13  dropped                88 non-null     object         
#  14  speech_header          88 non-null     object         
#  15  speech_flat_norepeat   88 non-null     object         
#  16  speech_final           88 non-null     object         
#  17  speech_final_expanded  88 non-null     object         
# dtypes: datetime64[ns](3), object(14), timedelta64[ns](1)
# memory usage: 12.5+ KB
# None

# ~~~df_speakers_combined~~~

# <class 'pandas.core.frame.DataFrame'>
# Int64Index: 34847 entries, 0 to 34846
# Data columns (total 10 columns):
#  #   Column           Non-Null Count  Dtype          
# ---  ------           --------------  -----          
#  0   corpus           34847 non-null  object         
#  1   exhibit          34847 non-null  object         
#  2   speaker_clean    34847 non-null  object         
#  3   nonspeech_clean  34847 non-null  object         
#  4   speech_final     34847 non-null  object         
#  5   location         34847 non-null  object         
#  6   date             34847 non-null  datetime64[ns] 
#  7   start_dtime      34847 non-null  datetime64[ns] 
#  8   end_dtime        34847 non-null  datetime64[ns] 
#  9   time_diff        34847 non-null  timedelta64[ns]
# dtypes: datetime64[ns](3), object(6), timedelta64[ns](1)
# memory usage: 2.9+ MB


~~~df_all_combined~~~

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   exhibit                88 non-null     object         
 1   corpus                 88 non-null     object         
 2   conversation_number    88 non-null     object         
 3   cassette_number        88 non-null     object         
 4   abstract               31 non-null     object         
 5   location               88 non-null     object         
 6   participants           88 non-null     object         
 7   minutes                88 non-null     object         
 8   txtdate                88 non-null     object         
 9   date                   88 non-null     datetime64[ns] 
 10  start_dtime            88 non-null     datetime64[ns] 
 11  end_dtime              88 non-null     datetime64[ns] 
 12  time_diff              88 no

### Store as Microsoft Excel Spreadsheets

In [30]:
# generate excel file for R script PreText analysis
path = "xlsx_combined_df_chunks.xlsx"
df_chunks_combined.to_excel(path)

## Build a pickle file for google collab upload
Reducing the size of the pickle file makes the upload much faster.

In [31]:
# Create a spreadsheet for coreference resolution in Google Collab
df_collab = df_chunks_combined[['corpus','speech_final','exhibit', 'orig_exhibit']].copy()
#df_collab.reset_index(inplace=True, drop=True)
df_collab.tail()

,corpus,speech_final,exhibit,orig_exhibit
136,WSPF,"No, I'm not going to bother with them. I ha...",Part_2_442-001-069,442-001-069
137,WSPF,"The, uh, FBI clips? I have that morning? They...",Part_3_442-001-069,442-001-069
138,WSPF,Good Which is very good. And then we take ou...,Part_4_442-001-069,442-001-069
139,WSPF,"Yes, please. Mr. Haldeman, please. Thank yo...",039-080_039-081,039-080_039-081
140,WSPF,Bob? Just a moment. Here you are. Sorry to...,039-083,039-083


In [32]:
# pickle reduced dataframe to reduce upload size
output = open('df_chunks_collab.pkl', 'wb')
sys.setrecursionlimit(100000)

pickle.dump(df_collab, output)

output.close()

In [33]:
# unpickle preserved dataframes to continue EDA and Statistical Analysis
pkl_file = open('df_chunks_collab.pkl', 'rb')

df_chunks_collab = pickle.load(pkl_file)
print("\n~~~df_chunks_collab~~~\n")
print(df_chunks_collab.info())


~~~df_chunks_collab~~~

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   corpus        141 non-null    object
 1   speech_final  141 non-null    object
 2   exhibit       141 non-null    object
 3   orig_exhibit  141 non-null    object
dtypes: object(4)
memory usage: 4.5+ KB
None


In [34]:
df_chunks_collab.tail()

,corpus,speech_final,exhibit,orig_exhibit
136,WSPF,"No, I'm not going to bother with them. I ha...",Part_2_442-001-069,442-001-069
137,WSPF,"The, uh, FBI clips? I have that morning? They...",Part_3_442-001-069,442-001-069
138,WSPF,Good Which is very good. And then we take ou...,Part_4_442-001-069,442-001-069
139,WSPF,"Yes, please. Mr. Haldeman, please. Thank yo...",039-080_039-081,039-080_039-081
140,WSPF,Bob? Just a moment. Here you are. Sorry to...,039-083,039-083


In [35]:
df_collab.to_excel("xlsx_df_collab.xlsx")  